# Illustrationen der Gartenlaube

Im Volltexttranskript der Gartenlaube sind Illustrationen freigestellt eingebettet. Dieses Skript analyisiert alle vorhandenen Artikeleinheiten in Wikisource und extrahiert freigestellte Gartenlaube-Illustrationien.

## 1. Extraktion der Illustrations-Dateinamen
Identifizierung von eingebetteten Illustrationen in Gartenlaube-Artikel via parse-API


## 2. Überprüfung ob die Datei in Commons bereits besteht
Bei der Artikel-Erstellung in Commons werden Illustrationen oft bereits "gesetzt" ohne, dass diese freigestellt in Commons gespeichert sind. (Darstellung als Rotlink) - Für die weitere Verarbeitung in diesem Skript ist aber notwendig, dass die Bilder physisch existieren - daher wie commons-Parse-API Überprüfung, ob die Datei existiert, wenn ja, dann wird die Medien-ID (pageid) und der Commons-File-Name für die Weiterverarbeitung übernommen.

## 3. Extraktion der Bildlegende aus Wikisource
Ein Großteil der Gartenlaube-Illustrationen ist mit einer Legende versehen. Dieser Textbaustein liegt in Wikisource nicht in der Artikel-Seite sondern im Seiten-Namespace des jeweiligen Scans, in der speziellen Wikisource-Vorlage ```{{BRU}}``` 


In [33]:
import requests
import re
import json
import urllib
import os
from SPARQLWrapper import SPARQLWrapper, JSON
import time
import sys
import wikitextparser as wtp

endpoint_url = "https://query.wikidata.org/sparql"

wdImgQSFile = 'output/img_WD_'+str(time.time())[0:10]+'.tsv'
sdcQSFile = 'output/img_SDC_'+str(time.time())[0:10]+'.tsv'
#QSFile = 'output/qs'+str(time.time())[0:10]+'.tsv'
try:
    os.remove(wdImgQSFile)
    os.remove(sdcQSFile)
except:
    pass

S = requests.Session()
artikelCounter = 1
resultSet = []

def getWikibaseIDviaMwapi(pagetitle,URL):
    param_wbase = { 'action' : 'query', 'prop' : 'pageprops', 'ppprop': 'wikibase_item',
             'redirects': 1, 'titles': pagetitle,'format': "json" }
    R = S.get(url=URL, params=param_wbase)
    DATA = R.json()
    try:
        for page in DATA["query"]["pages"]:
            #print(page)
            try:
                return DATA["query"]["pages"][page]["pageprops"]["wikibase_item"]
            except KeyError:
                pass
    except KeyError:
        pass

    
def getWikibaseIDviaSPARQL(pagetitle):
    query = """SELECT * WHERE {
      ?gartenlaubeArtikel wdt:P1433 wd:Q655617.
      ?article schema:about ?gartenlaubeArtikel.
      ?article schema:isPartOf <https://de.wikisource.org/>.
      ?article schema:name ?pageTitle.
      FILTER(STR(?pageTitle)='"""+str(pagetitle)+"""')
    }"""
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()
    #results = get_results(endpoint_url, query)
    #for result in results["results"]["bindings"]:
        #print(result["gartenlaubeArtikel"]["value"])

    
def getOldIDviaMwapi(pagetitle):
    param_wbase = { 'action' : 'query', 'prop' : 'info',
             'redirects': 1, 'titles': pagetitle,'format': "json" }
    R = S.get(url=URL, params=param_wbase)
    DATA = R.json()
    for page in DATA["query"]["pages"]:
        #print(page)
        try:
            return DATA["query"]["pages"][page]["lastrevid"]
        except KeyError:
            pass   

def parseWikiAPI(URL,PARAMS):
    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    return DATA


#Run through Category
def runthroughCat(DATA,count=0):
    try:
        while DATA['query']:
            #print(str(artikelCounter))
            #print(resultSet)
            global artikelCounter
            artikelCounter = ExtractImages(DATA,artikelCounter)

            try:
                #print(DATA['continue']['cmcontinue'])
                PARAMS['cmcontinue'] = DATA['continue']['cmcontinue']
                DATA = parseWikiAPI(URL,PARAMS)
            except KeyError:
                break
    except KeyError:
        pass


#Parse via Subcategories
def runthroughSubCat(kategorie):
    param_SubCat = PARAMS
    param_SubCat["cmtype"]="subcat"
    param_SubCat["cmtitle"] = kategorie
    DATACAT = parseWikiAPI(URL,param_SubCat)
    #print(DATACAT["query"])
    try:
        #print(str(artikelCounter))
        #print(resultSet)
        for subcat in DATACAT["query"]["categorymembers"]:
            print (">>=============================================")
            print (subcat["title"])
            PARAMS["cmtitle"] = subcat["title"]
            PARAMS.pop('cmtype',None)
            #print(PARAMS)
            DATA = parseWikiAPI(URL,PARAMS)
            runthroughCat(DATA)
    except KeyError:
        pass
    

def get_results(endpoint_url, query):
    user_agent = "Wikisource:Die_Gartenlaube Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()    


URL = "https://de.wikisource.org/w/api.php"
#?action=query&list=categorymembers&cmlimit=500&cmtitle=Kategorie:Album_der_S%C3%A4chsischen_Industrie&format=json
PARAMS = {
    'action': "query",
    'list': "categorymembers",
    'cmtitle': "Kategorie:Die_Gartenlaube_(1853)_Artikel",
    'cmlimit': 500,
    'format': "json"
}

def ExtractImages(DATA,artikelCounter):
    #print(DATA)
    for artikel in DATA["query"]["categorymembers"]:
        param_text = { 'action' : 'parse', 'page': artikel["title"], 'prop':'properties|images', 'format': "json" }
        R = S.get(url=URL, params=param_text)
        imgdata = R.json()

        qid = ""
        mid = ""
        for pprops in imgdata["parse"]["properties"]:
            if pprops["name"] == "wikibase_item":
                qid = pprops["*"]

        for image in imgdata["parse"]["images"]:
            f_wd = open(wdImgQSFile,'a')
            f_sdc = open(sdcQSFile,'a')
            #print(image)
            regex = r"Die_Gartenlaube_\(([\d]{4})\).*_b_([\d]{0,3}).*.jpg"
            matches = re.finditer(regex, image, re.MULTILINE)

            for matchNum, match in enumerate(matches, start=1):
                param_text = {'action' : 'parse', 'page': "File:"+image, 'prop':'images', 'format': "json" }
                R = S.get(url="http://commons.wikimedia.org/w/api.php", params=param_text)
                commonsdata = R.json()
                Das Pellerhaus in Nürnberg

                #print(commonsdata)
                try:
                    commonsTitle =re.sub("File:","",commonsdata["parse"]["title"])
                    mid = "M"+str(commonsdata["parse"]["pageid"] )
                except KeyError:
                    print("image "+image+" not in Commons yet")
                    break
                #print(image+" || page: "+match.group(2)+" || "+mid)
                #SDC Statement01 - published in 'GArtenlaube' generell
                print(mid+"\tP1433\tQ655617\tP304\t\""+match.group(2)+"\"\tP577\t+"+match.group(1)+"-00-00T00:00:00Z/9",file=f_sdc)
                #SDC Statement02 - published in -> Artikel-Item
                print(mid+"\tP1433\t"+qid,file=f_sdc)
                print(mid+"\tP1433\tQ655617\tP304\t\""+match.group(2)+"\"\tP577\t+"+match.group(1)+"-00-00T00:00:00Z/9")


                #Wikisource Seite 
                # https://de.wikisource.org/w/api.php?action=parse&page=Seite:Die_Gartenlaube_(1897)_476.jpg&format=json&prop=wikitext
                #extract aus BRU-Vorlage die Bildunterschrift
                seitenPage = "Seite:Die Gartenlaube ("+match.group(1)+") "+match.group(2)+".jpg"
                param_text = { 'action' : 'parse', 'page': seitenPage, 'prop':'wikitext', 'format': "json" }
                R = S.get(url=URL, params=param_text)
                wsSeitenData = R.json()
                parse = wtp.parse(wsSeitenData["parse"]["wikitext"]["*"])
                legende = ""
                for temp in parse.templates:

                    if temp.name == "BRU" and re.sub(" ","_",temp.arguments[0].value) == image:
                        #print(temp.arguments[0].value)

                        #print (temp.arguments[3].value)
                        '''
                        legende = re.sub("<br />"," ",temp.arguments[3].value)
                        legende = legende.replace('\n', ' ').replace('\r', '')
                        legende = re.sub("<br>"," ",legende)
                        '''
                        s = str(temp.arguments[3].value).replace('\n', ' ').replace('\r', '')
                        wtp.remove_markup(s)
                        legende = wtp.parse(s).plain_text()

                        if legende.strip() != "":
                            legendeQS = "\"\""+legende.strip()+"\"\" "
                        else:
                            legendeQS = ""
                        sdcDe =  legendeQS+"Illustration in: Die Gartenlaube ("+match.group(1)+"), S. "+match.group(2)
                        print(sdcDe)
                        #remove_markup(temp.arguments[3].value)
                        #SDC-Statement 03 -> deDescirptions ("caption")
                        print(mid+"\tLde\t\""+sdcDe+"\"",file=f_sdc)

                #query qid by given filename: https://w.wiki/36c$
                #nachschauen ob Bild bereits gesetzt, wenn nicht QID P18 Filname
                query = """SELECT * WHERE {
                      BIND(IRI(CONCAT("http://commons.wikimedia.org/wiki/Special:FilePath/", ENCODE_FOR_URI(\""""+re.sub("File:","",commonsdata["parse"]["title"])+"""\"))) AS ?img)
                      ?item p:P18 ?imgStmt. ?imgStmt ps:P18 ?img.
                    }"""
                #print(query)
                results = get_results(endpoint_url, query)
                if len(results["results"]["bindings"]) > 0:
                    for result in results["results"]["bindings"]:
                        print(result["item"]["value"])
                else:
                    if legende.strip() != "":
                        legendeStmt = "\tP2096\tde:\""+legende+"\""
                    else:
                        legendeStmt = ""
                    print(qid+"\tP18\t\""+commonsTitle+"\""+legendeStmt,file=f_wd)
                    print("image not linked in Wikidata-Item "+qid)
                print("==")
                #time.sleep(0.1)

DATA = parseWikiAPI(URL,PARAMS)
#print(DATA)
count = 0

R = S.get(url=URL, params=PARAMS)
DATA = R.json()


runthroughSubCat("Kategorie:Die_Gartenlaube_Artikel")



>>=============================================
Kategorie:Die Gartenlaube (1853) Artikel
M13102887	P1433	Q655617	P304	"397"	P577	+1853-00-00T00:00:00Z/9
""Alexander von Humboldt."" Illustration in: Die Gartenlaube (1853), S. 397
http://www.wikidata.org/entity/Q19125952
==
M13271474	P1433	Q655617	P304	"162"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 162
http://www.wikidata.org/entity/Q19186871
==
M16115186	P1433	Q655617	P304	"549"	P577	+1853-00-00T00:00:00Z/9
""Das Kapitol in Washington."" Illustration in: Die Gartenlaube (1853), S. 549
http://www.wikidata.org/entity/Q19127933
==
M16115192	P1433	Q655617	P304	"550"	P577	+1853-00-00T00:00:00Z/9
""Der Indianerhäuptling bei seiner Ankunft in Washington."" Illustration in: Die Gartenlaube (1853), S. 550
http://www.wikidata.org/entity/Q19127933
==
M16115196	P1433	Q655617	P304	"550"	P577	+1853-00-00T00:00:00Z/9
""Der Indianerhäuptling bei seinem Weggange von Washington."" Illustration in: Die Gartenlaube (1853), S

M12846139	P1433	Q655617	P304	"387"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 387
http://www.wikidata.org/entity/Q19157756
==
M13271602	P1433	Q655617	P304	"192"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 192
http://www.wikidata.org/entity/Q19215356
==
M13271608	P1433	Q655617	P304	"192"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 192
http://www.wikidata.org/entity/Q19215356
==
M16104045	P1433	Q655617	P304	"318"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 318
http://www.wikidata.org/entity/Q19215447
http://www.wikidata.org/entity/Q19143799
==
M16115041	P1433	Q655617	P304	"467"	P577	+1853-00-00T00:00:00Z/9
""Karte des russisch-türkischen Kriegsschauplatzes."" Illustration in: Die Gartenlaube (1853), S. 467
http://www.wikidata.org/entity/Q19148832
==
M13370387	P1433	Q655617	P304	"300"	P577	+1853-00-00T00:00:00Z/9
""Das Goldwaschen mit dem Longtom."" Illustration in: Die Gar

""Plesiosauren und Paläotherium."" Illustration in: Die Gartenlaube (1853), S. 537
http://www.wikidata.org/entity/Q19236357
==
M16104019	P1433	Q655617	P304	"289"	P577	+1853-00-00T00:00:00Z/9
""Nanking."" Illustration in: Die Gartenlaube (1853), S. 289
http://www.wikidata.org/entity/Q19180913
==
M13275932	P1433	Q655617	P304	"236"	P577	+1853-00-00T00:00:00Z/9
""Shakspeare’s Geburtshaus."" Illustration in: Die Gartenlaube (1853), S. 236
http://www.wikidata.org/entity/Q19192133
==
M13275956	P1433	Q655617	P304	"237"	P577	+1853-00-00T00:00:00Z/9
""Das Innere des Zimmers, in dem Shakspeare am 23. April 1564 geboren ist."" Illustration in: Die Gartenlaube (1853), S. 237
http://www.wikidata.org/entity/Q19192133
==
M13270313	P1433	Q655617	P304	"072"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 072
http://www.wikidata.org/entity/Q19153276
==
M13270328	P1433	Q655617	P304	"073"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 073
http://www.wikida

""Die tausend Inseln im Lawrencefluß."" Illustration in: Die Gartenlaube (1854), S. 581
http://www.wikidata.org/entity/Q19145226
==
M23184303	P1433	Q655617	P304	"224"	P577	+1854-00-00T00:00:00Z/9
http://www.wikidata.org/entity/Q19145970
==
M23125274	P1433	Q655617	P304	"095"	P577	+1854-00-00T00:00:00Z/9
""Joseph Bauer’s Dampfgrabemaschine."" Illustration in: Die Gartenlaube (1854), S. 095
http://www.wikidata.org/entity/Q19146323
==
M23184084	P1433	Q655617	P304	"159"	P577	+1854-00-00T00:00:00Z/9
""Das Industrie-Ausstellungs-Gebäude in München."" Illustration in: Die Gartenlaube (1854), S. 159
http://www.wikidata.org/entity/Q19182456
==
M23193652	P1433	Q655617	P304	"324"	P577	+1854-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1854), S. 324
http://www.wikidata.org/entity/Q66624849
==
M23193672	P1433	Q655617	P304	"325"	P577	+1854-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1854), S. 325
http://www.wikidata.org/entity/Q66624849
==
M23193686	P1433	Q655617	P304	"337"	P577	+1854-

http://www.wikidata.org/entity/Q19216405
==
M23184228	P1433	Q655617	P304	"201"	P577	+1854-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1854), S. 201
http://www.wikidata.org/entity/Q19216405
==
M23195514	P1433	Q655617	P304	"593"	P577	+1854-00-00T00:00:00Z/9
""Dawison."" Illustration in: Die Gartenlaube (1854), S. 593
http://www.wikidata.org/entity/Q19189166
==
M23184278	P1433	Q655617	P304	"211"	P577	+1854-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1854), S. 211
http://www.wikidata.org/entity/Q19158238
==
M23184155	P1433	Q655617	P304	"177"	P577	+1854-00-00T00:00:00Z/9
""Lord John Russel."" Illustration in: Die Gartenlaube (1854), S. 177
http://www.wikidata.org/entity/Q19218192
==
M23184291	P1433	Q655617	P304	"219"	P577	+1854-00-00T00:00:00Z/9
""Malta."" Illustration in: Die Gartenlaube (1854), S. 219
http://www.wikidata.org/entity/Q19218810
==
M23193743	P1433	Q655617	P304	"385"	P577	+1854-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1854), S. 385
http://www.wikidat

...

http://www.wikidata.org/entity/Q77917721
==
M32612045	P1433	Q655617	P304	"445"	P577	+1889-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1889), S. 445
http://www.wikidata.org/entity/Q19192559
http://www.wikidata.org/entity/Q19196518
==
M32629215	P1433	Q655617	P304	"880"	P577	+1889-00-00T00:00:00Z/9
""Ein neuer Weltbürger.Nach einem Gemälde von B. Vautier."" Illustration in: Die Gartenlaube (1889), S. 880
http://www.wikidata.org/entity/Q79147101
==
M85118356	P1433	Q655617	P304	"148"	P577	+1889-00-00T00:00:00Z/9
""Fig. 1. Sitzrad mit Daimlerschem Motor."" Illustration in: Die Gartenlaube (1889), S. 148
http://www.wikidata.org/entity/Q33060428
==
M85118599	P1433	Q655617	P304	"148"	P577	+1889-00-00T00:00:00Z/9
""Fig. 2. Konstruktion des Sitzrades."" Illustration in: Die Gartenlaube (1889), S. 148
http://www.wikidata.org/entity/Q33060428
==
M85118492	P1433	Q655617	P304	"148"	P577	+1889-00-00T00:00:00Z/9
""Fig. 3. Straßenwagen mit Daimlerschem Motor."" Illustration in: Die Gartenlaube (

""Schuhplattler. Nach einer Zeichnung von Schwabenmajer.Photographie im Verlage der „Photographischen Union“ in München."" Illustration in: Die Gartenlaube (1889), S. 397
http://www.wikidata.org/entity/Q19165139
==
M85521934	P1433	Q655617	P304	"573"	P577	+1889-00-00T00:00:00Z/9
""Massen-Freiübungen."" Illustration in: Die Gartenlaube (1889), S. 573
http://www.wikidata.org/entity/Q80791512
==
M85554402	P1433	Q655617	P304	"574"	P577	+1889-00-00T00:00:00Z/9
""Musterriegenturnen am Pferd."" Illustration in: Die Gartenlaube (1889), S. 574
http://www.wikidata.org/entity/Q80791512
==
M45737528	P1433	Q655617	P304	"577"	P577	+1889-00-00T00:00:00Z/9
""Der Festwagen im Zuge des Siebenten deutschen Turnfestes in München.Zeichnung von Fritz Bergen."" Illustration in: Die Gartenlaube (1889), S. 577
http://www.wikidata.org/entity/Q80791512
==
M85122906	P1433	Q655617	P304	"157"	P577	+1889-00-00T00:00:00Z/9
http://www.wikidata.org/entity/Q84294681
==
M85798732	P1433	Q655617	P304	"532"	P577	+1889-00-00T

M86068374	P1433	Q655617	P304	"717"	P577	+1889-00-00T00:00:00Z/9
""Fig. 5Nach innen und außen schlagendes Fenster."" Illustration in: Die Gartenlaube (1889), S. 717
image not linked in Wikidata-Item Q62616262
==
M86068590	P1433	Q655617	P304	"717"	P577	+1889-00-00T00:00:00Z/9
""Fig. 6Schutzvorrichtung für Schlosser"" Illustration in: Die Gartenlaube (1889), S. 717
image not linked in Wikidata-Item Q62616262
==
M86069241	P1433	Q655617	P304	"718"	P577	+1889-00-00T00:00:00Z/9
""Fig. 7  Fig. 8Vorrichtung gegen Sturz aus dem Fenster.Bei offenem Fenster.  Bei geschlossenem Fenster."" Illustration in: Die Gartenlaube (1889), S. 718
image not linked in Wikidata-Item Q62616262
==
M85652773	P1433	Q655617	P304	"772"	P577	+1889-00-00T00:00:00Z/9
""Menschliche Nasenformen."" Illustration in: Die Gartenlaube (1889), S. 772
http://www.wikidata.org/entity/Q81253820
==
M8248364	P1433	Q655617	P304	"413"	P577	+1889-00-00T00:00:00Z/9
""Der Eiffelthurm.Nach einer photographischen Aufnahme gezeichnet von C. K

http://www.wikidata.org/entity/Q75849498
==
M55036030	P1433	Q655617	P304	"664"	P577	+1892-00-00T00:00:00Z/9
""Pilzförmige Termitenbauten."" Illustration in: Die Gartenlaube (1892), S. 664
http://www.wikidata.org/entity/Q75849498
==
M55036082	P1433	Q655617	P304	"665"	P577	+1892-00-00T00:00:00Z/9
""Elsa-Fall, oberhalb des Edmin-Arnold-Falls."" Illustration in: Die Gartenlaube (1892), S. 665
http://www.wikidata.org/entity/Q75849498
==
M54902251	P1433	Q655617	P304	"227"	P577	+1892-00-00T00:00:00Z/9
""Die erste deutsche Postexpedition von Dar es Salaam nach Bukoba."" Illustration in: Die Gartenlaube (1892), S. 227
image not linked in Wikidata-Item Q64871452
==
M32670258	P1433	Q655617	P304	"036"	P577	+1892-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1892), S. 036
image not linked in Wikidata-Item Q19126806
==
M54869853	P1433	Q655617	P304	"613"	P577	+1892-00-00T00:00:00Z/9
""Originalzeichnung von R. Püttner."" Illustration in: Die Gartenlaube (1892), S. 613
image not linked in Wikidat

M54870298	P1433	Q655617	P304	"766"	P577	+1892-00-00T00:00:00Z/9
""Bechießung bei elektrischem Licht."" Illustration in: Die Gartenlaube (1892), S. 766
image not linked in Wikidata-Item Q62616429
==
M54870301	P1433	Q655617	P304	"767"	P577	+1892-00-00T00:00:00Z/9
""Vergrößerung einer mikroskopischen Depesche."" Illustration in: Die Gartenlaube (1892), S. 767
image not linked in Wikidata-Item Q62616429
==
M54870309	P1433	Q655617	P304	"767"	P577	+1892-00-00T00:00:00Z/9
""Absuchen eines Schlachtfeldes unter Anwendung des elektrischen Scheinwerfers."" Illustration in: Die Gartenlaube (1892), S. 767
image not linked in Wikidata-Item Q62616429
==
M54870333	P1433	Q655617	P304	"769"	P577	+1892-00-00T00:00:00Z/9
""Fahrt mit dem Fesselballon bei den französischen Manövern."" Illustration in: Die Gartenlaube (1892), S. 769
image not linked in Wikidata-Item Q62616429
==
M54869195	P1433	Q655617	P304	"333"	P577	+1892-00-00T00:00:00Z/9
""Die Enthüllung des Radetzky-Denkmals in Wien.Nach einer Zeichnung

In [35]:
import requests
import re
import time
import wikitextparser as wtp
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

S = requests.Session()
URL = "https://de.wikisource.org/w/api.php"
endpoint_url = "https://query.wikidata.org/sparql"

wdImgQSFile = 'output/img_WD_'+str(time.time())[0:10]+'.tsv'
sdcQSFile = 'output/img_SDC_'+str(time.time())[0:10]+'.tsv'
f_wd = open(wdImgQSFile,'w')
f_sdc = open(sdcQSFile,'w')

#Skizzen aus dem Imkerleben (alles fetig außer SDC)
#Das Pellerhaus in Nürnberg (keine Bilder freigstellt)
#Das Raimundtheater in Wien (1 Bild nicht in WD)
param_text = { 'action' : 'parse', 'page': "Das Pellerhaus in Nürnberg", 'prop':'properties|images', 'format': "json" }
R = S.get(url=URL, params=param_text)
imgdata = R.json()


# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

'''
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)
'''

for pprops in imgdata["parse"]["properties"]:
    if pprops["name"] == "wikibase_item":
        qid = pprops["*"]

for image in imgdata["parse"]["images"]:
    
    #f_sdc = open(sdcQSFile,'a')
    #print(image)
    regex = r"Die_Gartenlaube_\(([\d]{4})\).*_b_([\d]{0,3}).*.jpg"
    matches = re.finditer(regex, image, re.MULTILINE)

    for matchNum, match in enumerate(matches, start=1):
        param_text = {'action' : 'parse', 'page': "File:"+image, 'prop':'images', 'format': "json" }
        R = S.get(url="http://commons.wikimedia.org/w/api.php", params=param_text)
        commonsdata = R.json()
        #print(commonsdata)
        try:
            commonsTitle =re.sub("File:","",commonsdata["parse"]["title"])
            mid = "M"+str(commonsdata["parse"]["pageid"] )
        except KeyError:
            print("image "+image+" not in Commons yet")
            break
        #print(image+" || page: "+match.group(2)+" || "+mid)
        #SDC Statement01 - published in 'GArtenlaube' generell
        print(mid+"\tP1433\tQ655617\tP304\t\""+match.group(2)+"\"\tP577\t+"+match.group(1)+"-00-00T00:00:00Z/9",file=f_sdc)
        #SDC Statement02 - published in -> Artikel-Item
        print(mid+"\tP1433\t"+qid,file=f_sdc)
        print(mid+"\tP1433\tQ655617\tP304\t\""+match.group(2)+"\"\tP577\t+"+match.group(1)+"-00-00T00:00:00Z/9")
        

        #Wikisource Seite 
        # https://de.wikisource.org/w/api.php?action=parse&page=Seite:Die_Gartenlaube_(1897)_476.jpg&format=json&prop=wikitext
        #extract aus BRU-Vorlage die Bildunterschrift
        seitenPage = "Seite:Die Gartenlaube ("+match.group(1)+") "+match.group(2)+".jpg"
        param_text = { 'action' : 'parse', 'page': seitenPage, 'prop':'wikitext', 'format': "json" }
        R = S.get(url=URL, params=param_text)
        wsSeitenData = R.json()
        parse = wtp.parse(wsSeitenData["parse"]["wikitext"]["*"])
        legende = ""
        for temp in parse.templates:
            
            if temp.name == "BRU" and re.sub(" ","_",temp.arguments[0].value) == image:
                #print(temp.arguments[0].value)
                
                #print (temp.arguments[3].value)
                '''
                legende = re.sub("<br />"," ",temp.arguments[3].value)
                legende = legende.replace('\n', ' ').replace('\r', '')
                legende = re.sub("<br>"," ",legende)
                '''
                s = str(temp.arguments[3].value).replace('\n', ' ').replace('\r', '')
                wtp.remove_markup(s)
                legende = wtp.parse(s).plain_text()
                
                sdcDe = "\""+legende.strip()+"\" Illustration in: Die Gartenlaube ("+match.group(1)+"), S. "+match.group(2)
                print(sdcDe)
                #remove_markup(temp.arguments[3].value)
                #SDC-Statement 03 -> deDescirptions ("caption")
                print(mid+"\tDde\t\""+sdcDe+"\"",file=f_sdc)

        #query qid by given filename: https://w.wiki/36c$
        #nachschauen ob Bild bereits gesetzt, wenn nicht QID P18 Filname
        query = """SELECT * WHERE {
              BIND(IRI(CONCAT("http://commons.wikimedia.org/wiki/Special:FilePath/", ENCODE_FOR_URI(\""""+re.sub("File:","",commonsdata["parse"]["title"])+"""\"))) AS ?img)
              ?item p:P18 ?imgStmt. ?imgStmt ps:P18 ?img.
            }"""
        #print(query)
        results = get_results(endpoint_url, query)
        if len(results["results"]["bindings"]) > 0:
            for result in results["results"]["bindings"]:
                print(result["item"]["value"])
        else:
            if legende.strip() != "":
                legendeStmt = "\tP2096\tde:\""+legende+"\""
            else:
                legendeStmt = ""
            print(qid+"\tP18\t\""+commonsTitle+"\""+legendeStmt,file=f_wd)
            print("image not linked in Wikidata-Item "+qid)
        print("==")

print("",file=f_wd)

image Die_Gartenlaube_(1895)_b_124.jpg not in Commons yet
image Die_Gartenlaube_(1895)_b_125.jpg not in Commons yet
